#### Milestone 3 - Random Forest Test

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, isnan, count, when, isnull, size, split
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType, FloatType, DateType
from pyspark.sql.functions import col, regexp_replace
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
spark = SparkSession.builder.appName('final_project').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/12 02:53:41 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
24/04/12 02:53:41 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
24/04/12 02:53:41 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/12 02:53:41 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
# Put in your bucket and folder path without the csv file
BUCKET = 'gs://ds5460-tlee-spring2024/notebooks/jupyter//data/usa/combined_datasets/' 

data_with_review = spark.read.csv(BUCKET + 'real-final-data.csv',inferSchema=True,header=True)
data_with_zero_review = spark.read.csv(BUCKET + 'target_column_with_zero_review.csv',inferSchema=True,header=True)

In [4]:
# Double check the data was read in properly
data_with_review.head()

Row(host_total_listings_count=3.0, accommodates=8, num_bath=3.0, bedrooms=4.0, beds=6.0, price=500.0, amenities_count=14, essential_amenities=3, host_is_superhost='t', city='Broward County', room_type='Entire home/apt', full_time_host='f', host_verifications_clean='ep', target='Great')

In [5]:
# Double check the data was read in properly
data_with_zero_review.head()

24/04/12 02:54:01 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(id=827736378366911479, host_id=475630606, host_since='2022-08-18', host_location='Unknown', host_is_superhost='f', host_listings_count=1.0, host_total_listings_count=3.0, host_has_profile_pic='t', host_identity_verified='t', neighborhood='Fort Lauderdale', latitude=26.09393643124416, longitude=-80.13759087771177, room_type='Entire home/apt', accommodates=2, num_bath=1.0, bedrooms=1.0, beds=1.0, price=222.0, number_of_reviews=0, review_scores_value=None, calculated_host_listings_count=1, city='Broward County', amenities_count=10, neighborhood_city='Fort Lauderdale Broward County', full_time_host='f', host_verifications_clean='p', essential_amenities=3)

In [6]:
# Drop columns that can't be used in the feature pipeline as not applicable to zero reviews
data_with_zero_review = data_with_zero_review.drop('review_scores_value')
data_with_zero_review = data_with_zero_review.drop('number_of_reviews')

In [7]:
# Double check the columns in data_with_review
data_with_review.columns

['host_total_listings_count',
 'accommodates',
 'num_bath',
 'bedrooms',
 'beds',
 'price',
 'amenities_count',
 'essential_amenities',
 'host_is_superhost',
 'city',
 'room_type',
 'full_time_host',
 'host_verifications_clean',
 'target']

### Create the Feature Pipeline

In [8]:
# List of string features to be indexed and the features that are already numeric
stringFeatures = ['host_is_superhost', 'city', 'room_type', 'full_time_host', 'host_verifications_clean']

numericFeatures = ['host_total_listings_count', 'accommodates', 
                   'num_bath','bedrooms', 'beds', 'price',
                   'amenities_count', 'essential_amenities']

# Create StringIndexer stages for the stringFeatures - call the numeric version as _indexed
indexers = [StringIndexer(inputCol=column, outputCol=column+"_indexed", stringOrderType="alphabetAsc").setHandleInvalid("skip")
            for column in stringFeatures]

# Create a StringIndexer for the target column and naming it as target_label after converting it to numeric
labelIndexer = StringIndexer(inputCol='target', outputCol='target_label').setHandleInvalid("skip")

# Create VectorAssembler stage for the features
assemblerInputs = [column+"_indexed" for column in stringFeatures] + numericFeatures
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

# Create and run the pipeline with the target indexer added to pipeline
pipeline = Pipeline(stages=indexers + [assembler, labelIndexer])

# Fit the pipeline on the training data - which is all of data that has reviews
pipelineModel = pipeline.fit(data_with_review)

# Alternative code if we wanted to do the train/test split earlier
# trainingDataTransformed = pipelineModel.transform(trainingData)
# testDataTransformed = pipelineModel.transform(testData)

# Apply the pipeline on the data_with_review - trainingDataTransformed seen below will have to go through a train/test split
trainingDataTransformed = pipelineModel.transform(data_with_review)


# To apply the pipeline on the data_with_zero_review, which is missing the target, we need to do the following:
# Extracting the transformation stages from the fitted pipeline model
# Excldue the LabelIndexer from final pipeline stage as we are applying on the prediction dataset
transformationStages = pipelineModel.stages[:-1]

# Manually apply each transformation stage to the data_with_zero_review
transformedData = data_with_zero_review
for stage in transformationStages:
    transformedData = stage.transform(transformedData)

# We now have dataWithZeroReviewTransformed which doesn't have the target_label column to see what is our final model predictions
dataWithZeroReviewTransformed = transformedData

### Check if Feature Pipeline was done properly

In [9]:
trainingDataTransformed.printSchema()

root
 |-- host_total_listings_count: double (nullable = true)
 |-- accommodates: integer (nullable = true)
 |-- num_bath: double (nullable = true)
 |-- bedrooms: double (nullable = true)
 |-- beds: double (nullable = true)
 |-- price: double (nullable = true)
 |-- amenities_count: integer (nullable = true)
 |-- essential_amenities: integer (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- city: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- full_time_host: string (nullable = true)
 |-- host_verifications_clean: string (nullable = true)
 |-- target: string (nullable = true)
 |-- host_is_superhost_indexed: double (nullable = false)
 |-- city_indexed: double (nullable = false)
 |-- room_type_indexed: double (nullable = false)
 |-- full_time_host_indexed: double (nullable = false)
 |-- host_verifications_clean_indexed: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- target_label: double (nullable = false)



In [10]:
dataWithZeroReviewTransformed.printSchema()

root
 |-- id: long (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_since: string (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_listings_count: double (nullable = true)
 |-- host_total_listings_count: double (nullable = true)
 |-- host_has_profile_pic: string (nullable = true)
 |-- host_identity_verified: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- room_type: string (nullable = true)
 |-- accommodates: integer (nullable = true)
 |-- num_bath: double (nullable = true)
 |-- bedrooms: double (nullable = true)
 |-- beds: double (nullable = true)
 |-- price: double (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- amenities_count: integer (nullable = true)
 |-- neighborhood_city: string (nullable = true)
 |-- full_tim

In [11]:
trainingDataTransformed.show(n=1, truncate=False)

+-------------------------+------------+--------+--------+----+-----+---------------+-------------------+-----------------+--------------+---------------+--------------+------------------------+------+-------------------------+------------+-----------------+----------------------+--------------------------------+--------------------------------------------------------+------------+
|host_total_listings_count|accommodates|num_bath|bedrooms|beds|price|amenities_count|essential_amenities|host_is_superhost|city          |room_type      |full_time_host|host_verifications_clean|target|host_is_superhost_indexed|city_indexed|room_type_indexed|full_time_host_indexed|host_verifications_clean_indexed|features                                                |target_label|
+-------------------------+------------+--------+--------+----+-----+---------------+-------------------+-----------------+--------------+---------------+--------------+------------------------+------+-------------------------+---

In [12]:
# Doing a quick check that the string indexer worked and converted the categorical values into an index
trainingDataTransformed.select('host_verifications_clean', 'host_verifications_clean_indexed').show(5)

trainingDataTransformed.select('room_type', 'room_type_indexed').show(10)

+------------------------+--------------------------------+
|host_verifications_clean|host_verifications_clean_indexed|
+------------------------+--------------------------------+
|                      ep|                             1.0|
|                      pw|                             8.0|
|                      ep|                             1.0|
|                      ep|                             1.0|
|                      ep|                             1.0|
+------------------------+--------------------------------+
only showing top 5 rows

+---------------+-----------------+
|      room_type|room_type_indexed|
+---------------+-----------------+
|Entire home/apt|              0.0|
|Entire home/apt|              0.0|
|Entire home/apt|              0.0|
|Entire home/apt|              0.0|
|Entire home/apt|              0.0|
|Entire home/apt|              0.0|
|   Private room|              2.0|
|   Private room|              2.0|
|   Private room|              2.0|
|  

In [15]:
# Checking that the target_label got created successfully
trainingDataTransformed.select('target', 'target_label').show(20)

+-------+------------+
| target|target_label|
+-------+------------+
|  Great|         0.0|
|Average|         1.0|
|Average|         1.0|
|Average|         1.0|
|  Great|         0.0|
|Average|         1.0|
|  Great|         0.0|
|Average|         1.0|
|Average|         1.0|
|Average|         1.0|
|  Great|         0.0|
|Average|         1.0|
|  Great|         0.0|
|  Great|         0.0|
|Average|         1.0|
|Average|         1.0|
|   Poor|         2.0|
|  Great|         0.0|
|  Great|         0.0|
|Average|         1.0|
+-------+------------+
only showing top 20 rows



In [16]:
# Examine all of the features with a particular target_label
trainingDataTransformed.select("features", "target_label").show(truncate=False)

+-----------------------------------------------------------+------------+
|features                                                   |target_label|
+-----------------------------------------------------------+------------+
|[1.0,1.0,0.0,0.0,1.0,3.0,8.0,3.0,4.0,6.0,500.0,14.0,3.0]   |0.0         |
|[0.0,1.0,0.0,0.0,8.0,12.0,6.0,2.0,2.0,4.0,186.0,22.0,4.0]  |1.0         |
|[0.0,1.0,0.0,0.0,1.0,20.0,7.0,2.0,2.0,5.0,297.0,17.0,3.0]  |1.0         |
|[0.0,1.0,0.0,0.0,1.0,5.0,4.0,1.0,1.0,2.0,162.0,69.0,5.0]   |1.0         |
|[0.0,1.0,0.0,0.0,1.0,17.0,2.0,1.0,1.0,1.0,92.0,17.0,4.0]   |0.0         |
|[1.0,1.0,0.0,0.0,1.0,1930.0,6.0,2.0,2.0,4.0,258.0,38.0,5.0]|1.0         |
|[1.0,1.0,2.0,0.0,4.0,1.0,4.0,1.0,1.0,2.0,100.0,72.0,4.0]   |0.0         |
|[0.0,1.0,2.0,0.0,8.0,19.0,4.0,1.0,1.0,2.0,189.0,60.0,5.0]  |1.0         |
|[1.0,1.0,2.0,0.0,1.0,2.0,3.0,1.0,1.0,1.0,63.0,67.0,5.0]    |1.0         |
|[0.0,1.0,2.0,0.0,1.0,13.0,2.0,1.0,1.0,1.0,127.0,51.0,5.0]  |1.0         |
|[0.0,1.0,2.0,0.0,1.0,112

In [17]:
# Examine if the dataWithZeroReviewTransformed features look like it was done correctly
dataWithZeroReviewTransformed.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,2.0,0.0,...|
|[0.0,1.0,0.0,0.0,...|
|[0.0,1.0,2.0,0.0,...|
|[1.0,1.0,0.0,0.0,...|
+--------------------+
only showing top 5 rows



In [18]:
# See the count of each dataset to get an understanding on sizes:
num_rows = trainingDataTransformed.count()
num_columns = len(trainingDataTransformed.first())

print("For trainingDataTransformed:")
print(f"Number of Rows: {num_rows}, Number of Columns: {num_columns}")

num_rows = dataWithZeroReviewTransformed.count()
num_columns = len(dataWithZeroReviewTransformed.first())

print("For dataWithZeroReviewTransformed:")
print(f"Number of Rows: {num_rows}, Number of Columns: {num_columns}")

For trainingDataTransformed:
Number of Rows: 143995, Number of Columns: 21
For dataWithZeroReviewTransformed:
Number of Rows: 40989, Number of Columns: 31


## Random Forest Model

In [19]:
# Do the train test split - we need to discuss what are we settng for the threshold
train_data,test_data = trainingDataTransformed.randomSplit([0.7,0.3], seed=42)

In [20]:
rfc = RandomForestClassifier(labelCol='target_label',featuresCol='features', maxBins = 2200)

In [21]:
# Fit to the train_data
rfc_model = rfc.fit(train_data)

In [22]:
# Get the predictions for the test_data, which is the data with the ground truth known
rfc_predictions = rfc_model.transform(test_data)

In [23]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol="target_label", predictionCol="prediction", metricName="accuracy")

In [24]:
rfc_acc = acc_evaluator.evaluate(rfc_predictions)

In [25]:
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))

A random forest ensemble had an accuracy of: 57.95%


In [26]:
# Just seeing what is inside the rfc_predictions
rfc_predictions.select("features", "target_label", "rawPrediction", "probability", "prediction").show(truncate=False, n=2)

+--------------------------------------------------------+------------+---------------------------------------------------------+------------------------------------------------------------+----------+
|features                                                |target_label|rawPrediction                                            |probability                                                 |prediction|
+--------------------------------------------------------+------------+---------------------------------------------------------+------------------------------------------------------------+----------+
|[0.0,10.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,60.0,11.0,3.0]|0.0         |[10.493906030822693,7.4070682939240164,2.099025675253288]|[0.5246953015411348,0.37035341469620087,0.10495128376266442]|0.0       |
|[0.0,7.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,59.0,27.0,3.0] |0.0         |[10.955398166344272,7.528243835699906,1.5163579979558188]|[0.5477699083172137,0.3764121917849954,0.07581789989779096] |0.

In [27]:
# Applying the trained Random Forest model to the prediction dataset - the hold out zero reviews data
zeroReviewPredictions = rfc_model.transform(dataWithZeroReviewTransformed)

In [28]:
# Displaying predictions, probabilities, and features
zeroReviewPredictions.select("features", "rawPrediction", "probability", "prediction").show(truncate=False, n=5)

+----------------------------------------------------------+---------------------------------------------------------+-----------------------------------------------------------+----------+
|features                                                  |rawPrediction                                            |probability                                                |prediction|
+----------------------------------------------------------+---------------------------------------------------------+-----------------------------------------------------------+----------+
|[0.0,1.0,0.0,0.0,7.0,3.0,2.0,1.0,1.0,1.0,222.0,10.0,3.0]  |[10.268272811747906,7.588662116450266,2.14306507180183]  |[0.5134136405873952,0.3794331058225133,0.1071532535900915] |0.0       |
|[0.0,1.0,2.0,0.0,1.0,112.0,2.0,2.0,2.0,4.0,500.0,29.0,3.0]|[6.539864888062047,10.663418849222504,2.7967162627154503]|[0.32699324440310235,0.5331709424611252,0.1398358131357725]|1.0       |
|[0.0,1.0,0.0,0.0,4.0,6.0,16.0,4.0,6.0,12.0,868.0,

In [95]:
spark.stop()